In [3]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
import scipy.ndimage as img
import os 
import re
from astropy.io import fits
from skimage.transform import rotate, warp, SimilarityTransform
from math import sqrt
import math
import bottleneck as savior

In [89]:
obj = "KOA_4749"#"KOA_6004"#
cp = r"C:\Users\Tyler\Desktop"
#KOA_6004 = ROXs 12
#KOA_4749 = ROXs 42b

In [90]:
#path = r"C:\Users\Tyler\Desktop\{0}\NIRC2\raw\sci\{1}".format(obj, "N2.20110623.36186.jpg")#this path is for KOA_6004
path = r"C:\Users\Tyler\Desktop\{0}\NIRC2\raw\sci\{1}".format("KOA_4749", "N2.20110623.33237.jpg")
test = img.imread(path, mode = 'L')


In [72]:
plt.imshow(test,plt.cm.Greys_r)

plt.show()

In [73]:
def simpleCentroid(array,startPointTL, boxSize):
    x0,y0 = startPointTL
    cx,cy = 0,0
    totalFlux = 0
    i = 0
    while i < boxSize:
        j = 0
        while j < boxSize:
            f = array[x0+i][y0+j]
            cx+=i*f
            cy+=j*f
            totalFlux+=f
            j+=1
        i+=1
    return cx/totalFlux+x0, cy/totalFlux+y0


In [91]:
cx, cy = simpleCentroid(test, (606,546), 13)
print(cx,cy)
plt.imshow(test,cmap=plt.cm.Greys_r)
plt.plot(cx,cy,'rh')
plt.show()

611.517605634 552.27758216


In [92]:
#part 2
folderPath = r"C:\Users\Tyler\Desktop\{0}\NIRC2\raw\sci{1}".format(obj,"\\")


def positionRecorder(starName, path, startPoint, boxSize):
    inputs = list(filter(lambda s : s[-4:] == ".jpg", os.listdir(path)))
    output = [0]*len(inputs)
    for i in range(len(inputs)):
        image = img.imread(path+inputs[i], mode = 'L')
        cx, cy = simpleCentroid(image, startPoint, boxSize)
        output[i] = "{0},{1},{2}\n".format(cx,cy, inputs[i])
    with open('{0}_data.txt'.format(starName),'w') as f:
        for j in output:
            f.write(j)


In [93]:

positionRecorder(obj, folderPath, (606,546),13)

In [94]:
#part 3
def shifter(starName, filePath, picturePath):
    l = []
    with open(r'{0}\{1}_data.txt'.format(filePath,starName),'r') as f:
        l = list(iter(f.readline, ''))
    length = len(l)
    l[0] = re.split(",",l[0][:-1])
    x0, y0 = float(l[0][0]), float(l[0][1])
    l[0][0],l[0][1] = 0,0
    i = 1
    while i < length:
        l[i] = re.split(",",l[i][:-1])
        l[i][0], l[i][1] = float(l[i][0])-x0, float(l[i][1])-y0
        i+=1
    imageList = [0]*length
    nameList = [0]*length
    i = 0
    while i < length:
        imageList[i] = img.shift(img.imread(picturePath+l[i][-1], mode = 'L'),l[i][:2])
        nameList[i] = l[i][-1]
        i+=1
    return imageList, nameList, x0,y0
    
          
output, names, cx,cy = shifter(obj,r"C:\Users\Tyler\Desktop",folderPath)



In [95]:
outputStack = np.stack(output,axis=2)
imageSum = np.sum(outputStack,axis=2)
imageMedian = np.median(outputStack,axis=2)

In [13]:
plt.imshow(outputStack[:,:,43])
print(len(names))
plt.show()


46


In [332]:
plt.imshow(imageSum)
plt.show()

In [63]:
plt.imshow(imageMedian)
plt.show()

In [160]:
#before this use 7za.exe to batch decompress all the fits files
def rotateSystem(starName, filePath, picturePath):
    imageStack, names, cx,cy = shifter(starName, filePath, picturePath)
    cx, cy = int(cx), int(cy)
    #embed the arrays ahead of rotation
    size = np.shape(imageStack[0])[0]
    numImages = len(imageStack)
    middle = int(size*1.5)
    i = 0
    
    imageList = [0]*numImages
    while i < numImages:
        fitsI = fits.open(picturePath+names[i][:-4]+".fits")
        PA = fitsI[0].header['PARANG'] + fitsI[0].header['ROTPPOSN'] - \
        fitsI[0].header['EL'] - fitsI[0].header['INSTANGL']
        fitsI.close()
        #tempArray = rotate(imageStack[i], angle=PA,resize=True,cval=np.nan)#this is not an obvious package to use
        imageList[i] = rotate(imageStack[i],center=(cx,cy), order=3, angle=PA,resize=False,cval=np.nan, preserve_range=True)#this is not an obvious package to use
        i+=1
    return np.stack(imageList, axis=2), cx, cy



In [161]:
output2, cx,cy = rotateSystem(obj,r"C:\Users\Tyler\Desktop",folderPath)

C:\Anaconda2\envs\python3\lib\site-packages\astropy\io\fits\util.py:578: UserWarning: Could not find appropriate MS Visual C Runtime library or library is corrupt/misconfigured; cannot determine whether your file object was opened in append mode.  Please consider using a file object opened in write mode instead.
  'Could not find appropriate MS Visual C Runtime '


In [27]:
#plt.imshow(savior.nanmedian(output2,axis=2))
plt.imshow(savior.nanmedian(output2, axis=2), plt.cm.Greys_r)
#plt.plot(cx,cy,'rh')
plt.show()

In [120]:
plt.imshow(output2[:,:,1])
plt.show()

In [16]:
ff = range(np.shape(output2[:,:,0])[0])
whereMask = np.zeros(np.shape(output2)[:2])
for i in ff:
    whereMask[i,:] = list(map(lambda x: sqrt((x-cx)**2 + (i-cy)**2), ff))
    

In [53]:
plt.imshow(whereMask)
plt.show()
#np.shape(whereMask)

In [98]:
np.max(whereMask)#this also takes awhile

#part 5, this takes a little while, the time is primarily based on the dr you use
depth = np.shape(outputStack)[2]
dc = 0
step = 4
upperLimit = int(np.max(whereMask))
upperLimit -= upperLimit%step
newOut = [0]*depth
while dc < depth:
    counter = 0
    subtract = np.zeros(np.shape(outputStack[:,:,dc]))
    #print(np.shape(subtract))
    while upperLimit - counter > 0:
        counter +=step
        #print(dc)
        firstWhere = np.where((whereMask > counter-step) & (whereMask<=counter), output2[:,:,dc], np.nan)
        indices = np.where((whereMask > counter-step) & (whereMask<=counter))
        #print(type(indices[0]),type(indices[1]))

        gh = savior.nanmedian(firstWhere)
        subtract[indices[0],indices[1]] = gh
        
    newOut[dc]=outputStack[:,:,dc]-subtract
    dc+=1


In [67]:
plt.imshow(savior.nanmedian(np.stack(newOut,axis=2),axis=2))
plt.show()

In [99]:
#part 6
def rsImproved(starName, filePath, picturePath, whereMask):
    imageStack, names, cx,cy = shifter(starName, filePath, picturePath)
    
    
    upLimit = np.max(whereMask)
    medianList = [0]*(int(math.log(upLimit,2))+1)
    testList = [0]*(int(math.log(upLimit,2))+1)
    medianImg = np.median(np.stack(imageStack,axis=2),axis=2)
    counter = 1
    while counter < upLimit:
        counter*=2
        val = savior.nanmedian(np.where((whereMask > counter/2) & (whereMask<counter), medianImg, np.nan))
        medianList[int(math.log(counter,2))-1]  = val if val else 1
    
    #print(medianList)
    cx, cy = int(cx), int(cy)
    #embed the arrays ahead of rotation
    size = np.shape(imageStack[0])[0]
    numImages = len(imageStack)
    i = 0
    imageList = [0]*numImages
    while i < numImages:
        fitsI = fits.open(folderPath+names[i][:-4]+".fits")
        PA = fitsI[0].header['PARANG'] + fitsI[0].header['ROTPPOSN'] - \
        fitsI[0].header['EL'] - fitsI[0].header['INSTANGL']
        fitsI.close()
        #quick median calc
        counter = 1
        while counter < upLimit:
            counter*=2
            val = savior.nanmedian(np.where((whereMask > counter/2) & (whereMask<counter), imageStack[i], np.nan))
            testList[int(math.log(counter,2))-1] = val if val else 1
        
        ratios = [testList[j]/medianList[j] for j in range(len(testList))]
        medianScaleFactor = np.median(ratios)
        #meanScaleFactor = np.mean(ratios)
        imageList[i] = rotate(imageStack[i]-medianScaleFactor*medianImg,center=(cx,cy), order=3, angle=PA,resize=False,cval=np.nan, preserve_range=True)#this is not an obvious package to use
        i+=1
    return np.stack(imageList, axis=2), cx, cy

In [100]:
output6, cx,cy = rsImproved(obj,r"C:\Users\Tyler\Desktop",folderPath, whereMask)

C:\Anaconda2\envs\python3\lib\site-packages\astropy\io\fits\util.py:578: UserWarning: Could not find appropriate MS Visual C Runtime library or library is corrupt/misconfigured; cannot determine whether your file object was opened in append mode.  Please consider using a file object opened in write mode instead.
  'Could not find appropriate MS Visual C Runtime '


In [70]:
plt.imshow(output6[:,:,6])
plt.show()

In [68]:
plt.imshow(savior.nanmedian(output6,axis=2), plt.cm.Greys_r)

plt.show()

In [101]:
#part 7
def part7(obj, objList, commonPath, whereMask):#this assumes the obj is not part of the obj list
    imageStack, names, cx, cy = shifter(obj, commonPath,r"{0}\{1}\NIRC2\raw\sci\\".format(commonPath,obj))
    upLimit = np.max(whereMask)
    k = 0
    l0 = len(imageStack)
    nameList = [0]*l0
    best_cal = 0
    returnList = [0]*l0
    sciImageList = [0]*(int(math.log(upLimit,2))+1)
    calImageList = [0]*(int(math.log(upLimit,2))+1)
    while k < l0:
        i = 0
        l1 = len(objList)
        nameOfBest, chiBest = 0,1e10 
        counter = 1
        while counter < upLimit:
            counter*=2
            val = savior.nanmedian(np.where((whereMask > counter/2) & (whereMask<counter), imageStack[k], np.nan))
            sciImageList[int(math.log(counter,2))-1]  = val if val else 1
        while i < l1:
            psfComp, psfnames, ta1, ta2 = shifter(objList[i], commonPath, r"{0}\{1}\NIRC2\raw\sci\\".format(commonPath,objList[i]))
            j = 0
            l2 = len(psfComp)
            while j < l2:
                counter = 1
                while counter < upLimit:
                    counter*=2
                    val = savior.nanmedian(np.where((whereMask > counter/2) & (whereMask<counter), imageStack[i], np.nan))
                    calImageList[int(math.log(counter,2))-1] = val if val else 1
                scaleFactor = np.median([sciImageList[g]/calImageList[g] for g in range(len(sciImageList))])
                temp = imageStack[k] - scaleFactor*psfComp[j]
                chiTest = savior.ss(temp)
                if chiTest < chiBest:
                    chiBest = chiTest
                    nameOfBest = psfnames[j]
                    best_cal = temp
                j+=1
            i+=1
        nameList[k] = nameOfBest
        returnList[k] = best_cal
        k+=1
    with open(r'{0}_psfCal.txt'.format(obj),'w') as f:
        for i in range(len(names)):
            f.write("{0}, {1}\n".format(names[i],nameList[i]))
    return returnList
    

In [102]:
returnList = part7(obj,["KOA_6004"], cp, whereMask)#this takes awhile ~ 5 mins

In [105]:
plt.imshow(savior.nansum(np.stack(returnList, axis=2),axis=2))

plt.show()

In [103]:
#save all those images
plt.imsave(r"{0}\\{1}_part3sum.png".format(cp,obj), np.sum(np.stack(output,axis=2),axis=2),cmap = plt.cm.Greys)
plt.imsave(r"{0}\\{1}_part3median.png".format(cp,obj), np.median(np.stack(output,axis=2),axis=2),cmap = plt.cm.Greys)
plt.imsave(r"{0}\\{1}_part4median.png".format(cp,obj), savior.nanmedian(output2, axis=2),cmap = plt.cm.Greys)
plt.imsave(r"{0}\\{1}_part4sum.png".format(cp,obj), savior.nansum(output2, axis=2),cmap = plt.cm.Greys)
plt.imsave(r"{0}\\{1}_part5sum.png".format(cp,obj), savior.nansum(np.stack(newOut,axis=2),axis=2),cmap = plt.cm.Greys)
plt.imsave(r"{0}\\{1}_part5median.png".format(cp,obj), savior.nanmedian(np.stack(newOut,axis=2),axis=2),cmap=plt.cm.Greys)
plt.imsave(r"{0}\\{1}_part6median.png".format(cp,obj), savior.nanmedian(output6,axis=2),cmap=plt.cm.Greys)
plt.imsave(r"{0}\\{1}_part6sum.png".format(cp,obj), savior.nansum(output6,axis=2),cmap=plt.cm.Greys)
plt.imsave(r"{0}\\{1}_part7sum.png".format(cp,obj), savior.nansum(np.stack(returnList, axis=2),axis=2),cmap=plt.cm.Greys)
plt.imsave(r"{0}\\{1}_part7median.png".format(cp,obj), savior.nanmedian(np.stack(returnList, axis=2),axis=2),cmap=plt.cm.Greys)
plt.imsave(r"{0}\\{1}_whereMask.png".format(cp,obj), whereMask)

In [169]:
#part 8, assuming the same file naming as above
def meanPA(starName, filePath, picturePath):
    imageStack, names, cx,cy = shifter(starName, filePath, picturePath)
    numImages = len(imageStack)
    i = 0
    palist = [0]*numImages
    while i < numImages:
        fitsI = fits.open(picturePath+names[i][:-4]+".fits")
        PA = fitsI[0].header['PARANG'] + fitsI[0].header['ROTPPOSN'] - \
        fitsI[0].header['EL'] - fitsI[0].header['INSTANGL']
        fitsI.close()
        
        palist[i] = PA#this is not an obvious package to use
        i+=1
    return np.mean(palist)



obj = "KOA_6004"
guess ={"KOA_6004":[(662,382)], "KOA_4749":[(637,602),(721,594)]}
scale = 0.01#arcsec
index = 3
leng = len(guess[obj])
posAngleList = [[0]*5 for i in range(leng)]
projDist = [[0]*5 for i in range(leng)]
avgPA = meanPA(obj, cp, r"C:\Users\Tyler\Desktop\{0}\NIRC2\raw\sci{1}".format(obj,"\\"))
while index < 8:
    medianStack = plt.imread(r"{0}\\{1}_part{2}median.png".format(cp,obj,index))
    count=0
    while count < leng:
        x0,y0 = guess[obj][count]
        x0,y0 = x0-5, y0-5
        x,y = simpleCentroid(medianStack, (x0,y0),13)
        posAngleList[count][index-3] = 90-np.arctan2((cy-y),(cx-x))*180/np.pi + avgPA
        projDist[count][index-3] = scale*np.sqrt((cy-y)**2 + (cx-x)**2)
        count+=1 
    index+=1


C:\Anaconda2\envs\python3\lib\site-packages\astropy\io\fits\util.py:578: UserWarning: Could not find appropriate MS Visual C Runtime library or library is corrupt/misconfigured; cannot determine whether your file object was opened in append mode.  Please consider using a file object opened in write mode instead.
  'Could not find appropriate MS Visual C Runtime '


In [170]:
print(posAngleList,"\n", projDist, obj)

[[array([ 10.53779984,  10.53779984,  10.53779984,  10.53779984], dtype=float32), array([ 10.53786087,  10.53786087,  10.53786087,  10.53779984], dtype=float32), array([ 10.53779984,  10.53779984,  10.53779984,  10.53779984], dtype=float32), array([ 10.53779984,  10.53779984,  10.53779984,  10.53779984], dtype=float32), array([ 10.53779984,  10.53779984,  10.53779984,  10.53779984], dtype=float32)]] 
 [[array([ 1.76819122,  1.76819122,  1.76819122,  1.76819122], dtype=float32), array([ 1.76814187,  1.76814187,  1.76814187,  1.76819122], dtype=float32), array([ 1.76819122,  1.76819122,  1.76819122,  1.76819122], dtype=float32), array([ 1.76819122,  1.76819122,  1.76819122,  1.76819122], dtype=float32), array([ 1.76819122,  1.76819122,  1.76819122,  1.76819122], dtype=float32)]] KOA_6004


In [106]:
guess ={"KOA_6004":[(662,382)], "KOA_4749":[(637,602),(721,594)]}
guess["KOA_4749"]

[(637, 602), (721, 594)]

In [108]:
projDist = [[0]*4 for i in range(5)]

In [163]:
def meanPA(starName, filePath, picturePath):
    imageStack, names, cx,cy = shifter(starName, filePath, picturePath)
    numImages = len(imageStack)
    i = 0
    
    palist = [0]*numImages
    while i < numImages:
        fitsI = fits.open(picturePath+names[i][:-4]+".fits")
        PA = fitsI[0].header['PARANG'] + fitsI[0].header['ROTPPOSN'] - \
        fitsI[0].header['EL'] - fitsI[0].header['INSTANGL']
        fitsI.close()
        
        palist[i] = PA#this is not an obvious package to use
        i+=1
    return np.mean(palist)

16.008950434782609